In [3]:
                                                                                                                                                                                                                                                                                              # Cell 1: Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

from keras import layers, regularizers
import cv2
from keras import backend as K
import os
import glob
import re
import gc
from google.colab.patches import cv2_imshow
from keras.models import Sequential, Model
from keras.layers import (
    Dense,
    Dropout,
    Conv2D,
    Reshape,
    Input,
)
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [2]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print(physical_devices)
# if physical_devices:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
# Cell 2: Define Constants and Custom Loss
HEIGHT = 64
WIDTH = 64

def weighted_mse(yTrue, yPred):
    ones = K.ones_like(yTrue[0, :])
    idx = K.cumsum(ones)
    return K.mean((1 / idx) * K.square(yTrue - yPred))


In [5]:
# Cell 3: Image Manipulation Functions

# rotate image by a given angle
def rotateImage(image, angle):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    return cv2.warpAffine(image, M, (nW, nH))


In [ ]:
# Cell 4: Data Split Function
def splitter(train_ratio,validation_ratio):

    data=[]
    folders = glob.glob('/content/drive/MyDrive/PetImages/*') # Path to folder containing images
    for image_class in folders:
        # print(image_class)
        for image in os.listdir(os.path.join(image_class)):
            X=[]
            X.append(os.path.join(image_class,image));
            X.append(image_class.split('/')[-1])
            data.append(X)
    # # Split into train, validation, and test sets
    data=pd.Series(data)
    data=data.sample(frac=1) #randomizes the data array.
    train_size=int(train_ratio*len(data))
    train_files = data[0:train_size]
    validation_size=int(validation_ratio*len(train_files))
    validation_files = train_files[:validation_size]
    test_files = data[train_size:]


    # train_pca = []

    print(folders)
    print(len(train_files))
    print(len(validation_files))
    print(len(test_files))
    return train_files, test_files, validation_files

train_files, test_files, validation_files = splitter(train_ratio=0.7,validation_ratio=0.3)

In [ ]:
# Cell 5: PCA Function
def PCA_Done(train_files):
    data = np.zeros((len(train_files), 201))

    # for i, file in enumerate(train_files):
    #     label_name = # Get Labels

    # using the training files fit pca
    pca = PCA(n_components=20)
    pca.fit(data)
    explained_var = pca.explained_variance_ratio_
    print(explained_var)
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(explained_var)), explained_var, alpha=0.5, align='center', label='individual explained variance')
    plt.step(range(len(explained_var)), np.cumsum(explained_var), where='mid', label='cumulative explained variance')
    plt.xlabel('Principal components')
    plt.ylabel('Explained variance ratio')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

    print(sum(pca.explained_variance_ratio_))
    # print(pca)
    return pca
pca = PCA_Done(train_files)


In [31]:
# Cell 6: Batch Generator
def batch_generator(pca, X, batch_size=64):
    while True:
        # Select files (paths/indices) for the batch
        batch_paths = np.random.randint(low=0, high=len(X), size=batch_size)

        images = []
        props = []
        batch_label = []

        # Read in each input, perform preprocessing and get labels

        for index in batch_paths:
            img=cv2.imread(X[index][0])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img=cv2.resize(img,(64,64))
            img=img/255.0
            images.append(img)
            batch_label.append(X[index][1])

        # For Each file in the batch:
            # Add code for generating images and labels here.
            # First read the image as grayscale
            # Then resize the image to 64x64
            # Then normalize the image
            # Then reshape the image to (64,64,1)
            # Then append the image to the images list
            # Then read the corresponding label file
            # Then append the label to the labels list


        batch_x = np.array(images)
        batch_y = np.array(batch_label)

        yield (batch_x, batch_y)

train_gen = batch_generator(pca, train_files)
valid_gen = batch_generator(pca, validation_files, batch_size=8)
test_gen = batch_generator(pca, test_files, 1)

In [ ]:
# Cell 7: Model Building Function
def build_vit_model(image_size, num_classes, num_heads=4, num_transformer_layers=4):
    inputs = layers.Input(shape=(image_size, image_size, 3))
    # Convert image to patches
    patches = layers.Conv2D(filters=64, kernel_size=(8, 8), strides=(8, 8))(inputs)
    patches = layers.Reshape((64, 64))(patches)
    # Add Dropout
    patches = layers.Dropout(0.2)(patches)

    # Positional encoding
    pos_enc = layers.Embedding(input_dim=64, output_dim=64)(
        tf.range(start=0, limit=64, delta=1)
    )
    patches += pos_enc

    # Transformer layers
    for _ in range(num_transformer_layers):
        # Multi-head attention
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=64)(
            patches, patches
        )
        patches = layers.LayerNormalization()(patches + attn_output)

        # Feed-forward network
        ffn = layers.Dense(128, activation="relu")(patches)
        ffn = layers.Dense(64)(ffn)
        # add dropout
        ffn = layers.Dropout(0.2)(ffn)

        patches = layers.LayerNormalization()(patches)



    output = layers.Dense(20, activation="linear")(patches[:, 0, :])

    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

model = build_vit_model(64, 1)  # Replace 256 with your actual image size
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss=weighted_mse,
    metrics=["mae"],
)
print(model.summary())
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "Best_Model.h5", save_weights_only=True, save_best_only=True, mode="min"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.8,
        patience=100,
        min_lr=1e-5,
        min_delta=0.000001,
        verbose=1,
        mode="min",
    )
]

In [ ]:

# CODE FOR HYPERPARAMETERS TUNING

# from kerastuner.tuners import RandomSearch

# def build_model(hp):
#     model = build_vit_model(
#         image_size=hp.Int('image_size', min_value=32, max_value=256, step=32),
#         num_classes=10,
#         num_heads=hp.Int('num_heads', min_value=2, max_value=8, step=2),
#         num_transformer_layers=hp.Int('num_transformer_layers', min_value=1, max_value=8, step=1)
#     )
#     model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model

# tuner = RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=5,  # number of different hyperparameter configurations to try
#     executions_per_trial=3,  # number of times to train each model, to average out the metrics
#     directory='random_search',
#     project_name='vit'
# )

# tuner.search(train_gen, epochs=5, validation_data=(valid_gen))

In [ ]:
# best_model = tuner.get_best_models(num_models=1)[0]
# best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Cell 8: Prediction Function
def predict():
    for file in test_files[:25]:
        img = cv2.imread(file, 0)

        folder = os.path.split(os.path.split(file)[0])[1]

        # Image processing should be done here


        # Get Properties

        # Concatenate Image and Properties
        # img = np.concatenate((img, props), axis=2)
        # img = img.reshape(-1, img.shape[0], img.shape[1], img.shape[2])

        # Get Labels

        pred = model.predict(img)
        predictions = pca.inverse_transform(pred).reshape(-1)

        # label_file["pred_values"] = predictions
        # print(label_file["pred_values"].shape)
        # plt.ylim(0,0.1)
        # plt.plot(predictions)
        # plt.plot(label)
        # plt.legend(["Predictions", "Actual_Values"])
        # plt.show()

K.clear_session()
gc.collect()

In [ ]:
# model.load_weights('best_model_v2.h5')

history = model.fit(
    train_gen,
    epochs=1500,
    verbose=1,
    batch_size=32,
    validation_data=valid_gen,
    validation_steps=len(validation_files) // 8,
    steps_per_epoch=len(train_files) // 32,
    callbacks=callbacks,
)

model.save_weights('Final_Model_weights.h5')

model.save('Final_Model.h5')

In [ ]:
# plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.load_weights('Iteration_1_model.h5')
print(f"Test Loss: {model.evaluate(test_gen, steps=len(test_files))}")

In [ ]:
predict()